 Copyright © Sorbonne University.

 This source code is licensed under the MIT license found in the LICENSE file
 in the root directory of this source tree.

# Outlook

In this notebook we code one version of the [Proximal Policy Optimization
(PPO)](https://arxiv.org/pdf/1707.06347.pdf) algorithms using BBRL. More
precisely, the version here is the one that uses the KL penalty as a
regularization term when optimizing the policy gradient.

The PPO algorithm is superficially explained in [this
video](https://www.youtube.com/watch?v=uRNL93jV2HE) and you can also read [the
corresponding slides](http://pages.isir.upmc.fr/~sigaud/teach/ps/10_ppo.pdf).

It is also a good idea to have a look at the [spinning up
documentation](https://spinningup.openai.com/en/latest/algorithms/ppo.html).

This version of PPO works, but it incorrectly samples minibatches randomly
from the rollouts without making sure that each sample is used once and only
once See:
https://iclr-blog-track.github.io/2022/03/25/ppo-implementation-details/ for a
full description of all the coding tricks that should be integrated

# 展望

在这个笔记本中，我们使用 BBRL 编写了 [Proximal Policy Optimization (PPO)](https://arxiv.org/pdf/1707.06347.pdf) 算法的一个版本。具体来说，这个版本在优化策略梯度时使用 KL 惩罚作为正则项。

可以通过 [这个视频](https://www.youtube.com/watch?v=uRNL93jV2HE) 了解 PPO 算法的简介，并阅读 [对应的幻灯片](http://pages.isir.upmc.fr/~sigaud/teach/ps/10_ppo.pdf)。

还建议查看 [spinning up 文档](https://spinningup.openai.com/en/latest/algorithms/ppo.html) 。

这个 PPO 版本有效，但在从回合中随机抽取小批量样本时存在缺陷，详见 [文章](https://iclr-blog-track.github.io/2022/03/25/ppo-implementation-details/)

# Setting up the environment
We first need to setup the environment
Installs the necessary Python and system libraries

In [1]:
try:
    from easypip import easyimport
except ModuleNotFoundError:
    from subprocess import run
    assert run(["pip", "install", "easypip"]).returncode == 0, "Could not install easypip"
    from easypip import easyimport

easyimport("swig")
easyimport("bbrl_utils").setup()

import copy
import os

import torch
import torch.nn as nn
from bbrl.agents import Agent, Agents, KWAgentWrapper, TemporalAgent
from bbrl_utils.algorithms import EpisodicAlgo, iter_partial_episodes
from bbrl_utils.nn import build_ortho_mlp, setup_optimizer
from bbrl_utils.notebook import setup_tensorboard
from omegaconf import OmegaConf

from bbrl_utils.nn import copy_parameters

/home/chen_guanyu/deepdac/lib/python3.10/site-packages/bbrl_utils/notebook.py:46: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm  # noqa: F401
error: XDG_RUNTIME_DIR not set in the environment.


# Learning environment

## Configuration

The learning environment is controlled by a configuration that define a few
important things as described in the example below. This configuration can
hold as many extra information as you need, the example below is the minimal
one.

# 学习环境

## 配置

学习环境由一个配置文件控制，该文件定义了一些重要的元素，如以下示例所示。该配置可以包含所需的任意额外信息，下面的示例展示了最小配置。

```python
params = {
    # This defines the a path for logs and saved models
    "base_dir": "${gym_env.env_name}/myalgo_${current_time:}",

    # The Gymnasium environment
    "gym_env": {
        "env_name": "CartPoleContinuous-v1",
    },

    # Algorithm
    "algorithm": {
        # Seed used for the random number generator
        "seed": 1023,

        # Number of parallel training environments
        "n_envs": 8,
                
        # Minimum number of steps between two evaluations
        "eval_interval": 500,
        
        # Number of parallel evaluation environments
        "nb_evals": 10,

        # Number of epochs (loops)
        "max_epochs": 40000,

        # Number of steps (partial iteration)
        "n_steps": 100,
        
    },
}

# Creates the configuration object, i.e. cfg.algorithm.nb_evals is 10
cfg = OmegaConf.create(params)
```

## The RL algorithm

In this notebook, the RL algorithm is based on `EpisodicAlgo`, that defines
the algorithm environment when using episodes. To use such environment, we
just need to subclass `EpisodicAlgo` and to define two things, namely the
`train_policy` and the `eval_policy`. Both are BBRL agents that, given the
environment state, select the action to perform.

## 强化学习算法

在此笔记本中，强化学习算法基于 `EpisodicAlgo`，它定义了在使用回合时的算法环境。要使用这种环境，我们只需继承 `EpisodicAlgo` 并定义两个部分，即 `train_policy` 和 `eval_policy`。这两个部分都是 BBRL 代理，基于环境的状态选择要执行的动作。

```py
  class MyAlgo(EpisodicAlgo):
      def __init__(self, cfg):
          super().__init__(cfg)

          # Define the train and evaluation policies
          # (the agents compute the workspace `action` variable)
          self.train_policy = MyPolicyAgent(...)
          self.eval_policy = MyEvalAgent(...)

algo = MyAlgo(cfg)
```

The `EpisodicAlgo` defines useful objects:

- `algo.cfg` is the configuration
- `algo.nb_steps` (integer) is the number of steps since the training began
- `algo.logger` is a logger that can be used to collect statistics during training:
    - `algo.logger.add_log("critic_loss", critic_loss, algo.nb_steps)` registers the `critic_loss` value on tensorboard
- `algo.evaluate()` evaluates the current `eval_policy` if needed, and keeps the
agent if it was the best so far (average cumulated reward);
- `algo.visualize_best()` runs the best agent on one episode, and displays the video



Besides, it also defines an `iter_episodes` that allows to iterate over partial
episodes (with `n_steps` from `n_envs` environments):


```python3
  # with partial episodes
  for workspace in algo.iter_partial_episodes():
      # workspace is a workspace containing 50 transitions
      # (with autoreset)
      ...
```

`EpisodicAlgo` 定义了一些有用的对象：

- `algo.cfg`: 配置文件。
- `algo.nb_steps` (整数): 表示训练开始以来的步数。
- `algo.logger`: 日志记录器，用于在训练过程中收集统计数据，例如 `algo.logger.add_log("critic_loss", critic_loss, algo.nb_steps)` 用于将 `critic_loss` 值记录到 tensorboard。
- `algo.evaluate()`: 如有需要，评估当前的 `eval_policy`，并保存表现最佳的代理（根据平均累积奖励）。
- `algo.visualize_best()`: 在一回合中运行最佳代理并显示视频。

此外，还定义了一个 `iter_episodes`，允许在部分回合中迭代（基于 `n_steps` 和 `n_envs`）。

```python3
  # with partial episodes
  for workspace in algo.iter_partial_episodes():
      # workspace is a workspace containing 50 transitions
      # (with autoreset)
      ...
```

## Definition of PPO agents

### Critic agent

As A2C, PPO uses a value function $V(s)$. We thus call upon the `VAgent`
class,  which takes an observation as input and whose output is the value of
this observation.

## 定义PPO代理

### 价值网络代理（Critic agent）

和A2C算法一样，PPO使用一个值函数 $V(s)$。因此，我们使用`VAgent`类，它以观察值为输入，并输出该观察值对应的价值。

In [2]:
class VAgent(Agent):
    def __init__(self, state_dim, hidden_layers, name="critic"):
        super().__init__(name=name)
        self.is_q_function = False
        self.model = build_ortho_mlp(
            [state_dim] + list(hidden_layers) + [1], activation=nn.ReLU()
        )

    def forward(self, t, **kwargs):
        observation = self.get(("env/env_obs", t))
        critic = self.model(observation).squeeze(-1)
        self.set((f"{self.prefix}v_values", t), critic)

### KL penalty agent

When computing the KL penalty, we need to compute the KL divergence at every
time step. The KLAgent is specific to the KL regularization version of PPO. It
is used to compute the KL divergence between the current and the past policy.

### KL惩罚代理

在计算KL惩罚时，我们需要在每个时间步计算KL散度。`KLAgent`是PPO中KL正则化版本的特定代理，用于计算当前策略与过去策略之间的KL散度。

In [3]:
class KLAgent(Agent):
    def __init__(self, model_1, model_2):
        super().__init__()
        self.model_1 = model_1
        self.model_2 = model_2

    def forward(self, t, **kwargs):
        obs = self.get(("env/env_obs", t))
        dist_1 = self.model_1.dist(obs)
        dist_2 = self.model_2.dist(obs)
        kl = torch.distributions.kl.kl_divergence(dist_1, dist_2)
        self.set(("kl", t), kl)

### The DiscretePolicy

The DiscretePolicy was already used in A2C to deal with discrete actions, but
we have added the possibility to only predict the probability of an action
using the ```predict_proba``` variable in the ```forward()``` function. The
code is as follows.

### DiscretePolicy

`DiscretePolicy`之前已用于A2C以处理离散动作，但我们在此基础上增加了仅预测动作概率的功能。通过在`forward()`函数中使用`predict_proba`变量，可以实现这个功能。

In [4]:
class DiscretePolicy(Agent):
    def __init__(self, state_dim, hidden_size, n_actions, name="policy"):
        super().__init__(name=name)
        self.model = build_ortho_mlp(
            [state_dim] + list(hidden_size) + [n_actions], activation=nn.ReLU()
        )

    def dist(self, obs):
        scores = self.model(obs)
        probs = torch.softmax(scores, dim=-1)
        return torch.distributions.Categorical(probs)

    def forward(
        self,
        t,
        *,
        stochastic=True,
        predict_proba=False,
        compute_entropy=False,
        **kwargs,
    ):
        """
        Compute the action given either a time step (looking into the workspace)
        or an observation (in kwargs)
        """
        observation = self.get(("env/env_obs", t))
        scores = self.model(observation)
        probs = torch.softmax(scores, dim=-1)
        #print(probs)

        if predict_proba:
            action = self.get(("action", t))
            log_probs = probs[torch.arange(probs.size()[0]), action].log()
            self.set((f"{self.prefix}logprob_predict", t), log_probs)
        else:
            if stochastic:
                action = torch.distributions.Categorical(probs).sample()
            else:
                action = scores.argmax(1)

            self.set(("action", t), action)

        if compute_entropy:
            entropy = torch.distributions.Categorical(probs).entropy()
            self.set((f"{self.prefix}entropy", t), entropy)

### Main PPO agent

In the following, we create the PPO Agent, with one policy and one critic,
and their "delayed" versions (target network for the critic, and previous 
policy in the inner loop of the optimization).

### 主 PPO 代理

接下来，我们将创建主 PPO 代理，包含一个策略和一个价值函数（critic），以及它们的“延迟”版本（延迟网络，用于critic的目标网络和优化内部循环中的旧策略）。

In [5]:
class PPOPenalty(EpisodicAlgo):
    def __init__(self, cfg):
        super().__init__(cfg, autoreset=True)

        obs_size, act_size = self.train_env.get_obs_and_actions_sizes()
        self.train_policy = globals()[cfg.algorithm.policy_type](
            obs_size,
            cfg.algorithm.architecture.actor_hidden_size,
            act_size,
        ).with_prefix("current_policy/")
        self.eval_policy = KWAgentWrapper(
            self.train_policy, 
            stochastic=False,
            predict_proba=False,
            compute_entropy=False,
        )

        self.critic_agent = VAgent(
            obs_size, cfg.algorithm.architecture.critic_hidden_size
        ).with_prefix("critic/")
        self.old_critic_agent = copy.deepcopy(self.critic_agent).with_prefix("old_critic/")
        self.t_all_critics = TemporalAgent(
            Agents(self.critic_agent, self.old_critic_agent)
        )

        self.old_policy = copy.deepcopy(self.train_policy)
        self.old_policy.with_prefix("old_policy/")
        
        self.t_kl_agent = TemporalAgent(KLAgent(self.old_policy, self.train_policy))

        self.policy_optimizer = setup_optimizer(
            cfg.optimizer, self.train_policy
        )
        self.critic_optimizer = setup_optimizer(
            cfg.optimizer, self.critic_agent
        )

### Main PPO loop

In the cell below, we optimize the policy loss for PPO-KL, i.e.

### 主 PPO 循环

在下面的单元中，我们针对PPO-KL优化策略损失，即

$$
\max_{\theta} \hat{\mathbb{E}} _t\left[\frac{\pi_\theta\left(a_t \mid s_t\right)}{\pi_{\theta_{\text {old }}}\left(a_t \mid s_t\right)} \hat{A}_t-\beta \operatorname{KL}\left[\pi_{\theta_{\text {old }}}\left(\cdot \mid s_t\right), \pi_\theta\left(\cdot \mid s_t\right)\right]\right]
$$

In [6]:
from bbrl.utils.functional import gae


def run_ppo_penalty(ppo: PPOPenalty):
    cfg = ppo.cfg

    # The old_policy params must be wrapped into a TemporalAgent
    t_old_policy = TemporalAgent(ppo.old_policy)

    # Training loop
    for train_workspace in ppo.iter_partial_episodes():
        # Run the current policy and evaluate the proba of its action according
        # to the old policy The old_policy can be run after the train_agent on
        # the same workspace because it writes a logprob_predict and not an
        # action. That is, it does not determine the action of the old_policy,
        # it just determines the proba of the action of the current policy given
        # its own probabilities

        # Compute the critic value over the whole workspace
        ppo.t_all_critics(train_workspace, t=0, n_steps=cfg.algorithm.n_steps)

        ws_terminated, ws_reward, ws_v_value, ws_old_v_value = train_workspace[
            "env/terminated",
            "env/reward",
            "critic/v_values",
            "old_critic/v_values",
        ]

        # --- Critic optimization

        # Avoids to extreme V-values (helps stability)
        if cfg.algorithm.clip_range_vf > 0:
            # Clip the difference between old and new values
            # NOTE: this depends on the reward scaling
            ws_v_value = ws_old_v_value + torch.clamp(
                ws_v_value - ws_old_v_value,
                -cfg.algorithm.clip_range_vf,
                cfg.algorithm.clip_range_vf,
            )

        # Compute the advantage using the (clamped) critic values
        with torch.no_grad():
            advantage = gae(
                ws_reward[1:],
                ws_v_value[1:],
                ~ws_terminated[1:],
                ws_v_value[:-1],
                cfg.algorithm.discount_factor,
                cfg.algorithm.gae,
            )
        
        # Compute the critic loss with TD(0)
        target = ws_reward[1:] + cfg.algorithm.discount_factor * ws_old_v_value[1:].detach() * (1 - ws_terminated[1:].int())
        critic_loss = torch.nn.functional.mse_loss(ws_v_value[:-1], target) * cfg.algorithm.critic_coef
        ppo.critic_optimizer.zero_grad()
        critic_loss.backward()
        torch.nn.utils.clip_grad_norm_(
            ppo.critic_agent.parameters(), cfg.algorithm.max_grad_norm
        )
        ppo.critic_optimizer.step()
        
        # --- Policy optimization

        # We store the advantage into the train_workspace
        if cfg.algorithm.normalize_advantage and advantage.shape[1] > 1:
            advantage = (advantage - advantage.mean()) / (advantage.std() + 1e-8)
        train_workspace.set_full("advantage", torch.cat(
            (advantage, torch.zeros(1, advantage.shape[1]))
        ))

        with torch.no_grad():
            # Just computes the probability of the old policy's action
            # to get the ratio of probabilities
            t_old_policy(
                train_workspace,
                t=0,
                n_steps=cfg.algorithm.n_steps,
                predict_proba=True,
                compute_entropy=False,
            )

        transition_workspace = train_workspace.get_transitions()
        for opt_epoch in range(cfg.algorithm.opt_epochs):
            if cfg.algorithm.batch_size > 0:
                sample_workspace = transition_workspace.select_batch_n(
                    cfg.algorithm.batch_size
                )
            else:
                sample_workspace = transition_workspace
            
            # Compute the policy loss

            # Compute the KL divergence

            # 使用 KLAgent 计算 KL 散度
            ppo.t_kl_agent(sample_workspace,t=0,n_steps=2) 
            # 从 workspace 中提取 KL 散度
            kl = sample_workspace["kl"]

            # Compute the probability of the played actions according to the current policy
            # 计算当前策略的动作概率
            """ t_current_policy = TemporalAgent(ppo.train_policy)
            t_current_policy(
                sample_workspace,
                t=0,
                n_steps=2,
                predict_proba=True,
                compute_entropy=True
            ) """
            ppo.train_policy(
                sample_workspace,
                t=0,
                n_steps=2,
                predict_proba=True,
                compute_entropy=True
            )

            log_prob_current = sample_workspace["current_policy/logprob_predict"]

            # We do not replay the action: we use the one stored into the dataset
            # Note that the policy is not wrapped into a TemporalAgent, but we use a single step
            # 获取旧策略的动作概率
            log_prob_old = sample_workspace["old_policy/logprob_predict"]

            # Compute the ratio of action probabilities
            ratio = (log_prob_current - log_prob_old).exp()
            
            # Compute the policy loss
            # assert False, 'Not implemented yet'
            # 计算 PPO 策略损失
            policy_advantage = sample_workspace["advantage"]
            beta = cfg.algorithm.beta
            policy_loss = torch.mean(ratio * policy_advantage - beta * kl)

            loss_policy = -cfg.algorithm.policy_coef * policy_loss

            # Entropy loss favors exploration
            entropy = sample_workspace["current_policy/entropy"]
            entropy = torch.mean(entropy, dim=0, keepdim=True)
            
            # Note that the standard PPO algorithms do not have an entropy term, they don't need it
            # because the KL term is supposed to deal with exploration
            # So, to run the standard PPO algorithm, you should set cfg.algorithm.entropy_coef=0
            assert len(entropy) == 1, f"{entropy.shape}"
            entropy_loss = entropy[0].mean()
            loss_entropy = -cfg.algorithm.entropy_coef * entropy_loss

            # Store the losses for tensorboard display
            ppo.logger.log_losses(critic_loss, entropy_loss, policy_loss, ppo.nb_steps)
            ppo.logger.add_log("advantage", policy_advantage.mean(), ppo.nb_steps)

            ppo.policy_optimizer.zero_grad()
            loss = loss_policy + loss_entropy
            loss.backward()
            torch.nn.utils.clip_grad_norm_(
                ppo.train_policy.parameters(), cfg.algorithm.max_grad_norm
            )
            ppo.policy_optimizer.step()

        # Copy parameters
        copy_parameters(ppo.train_policy, ppo.old_policy)
        copy_parameters(ppo.critic_agent, ppo.old_critic_agent)
        
        # Evaluate
        ppo.evaluate()

## Definition of the parameters

In [7]:
params = {
    "base_dir": "${gym_env.env_name}/ppo-kl-S${algorithm.seed}-b${algorithm.beta}_${current_time:}",
    "save_best": False,
    "algorithm": {
        "seed": 12,
        "n_envs": 8,
        "n_steps": 32,
        "normalize_advantage": False,
        "eval_interval": 1000,
        "nb_evals": 10,
        "gae": 0.8,
        "discount_factor": 0.98,
        "opt_epochs": 10,
        "batch_size": 256,
        "max_grad_norm": 0.5,
        "policy_coef": 1.,
        "beta": 5.0,
        "clip_range_vf": 0,
        "entropy_coef": 2e-7,
        "critic_coef": 1.0,
        "policy_type": "DiscretePolicy",
        "architecture": {
            "actor_hidden_size": [64, 64],
            "critic_hidden_size": [64, 64],
        },
        "max_epochs": 5000,
    },
    "gym_env": {
        "env_name": "CartPole-v1",
    },
    "optimizer": {
        "classname": "torch.optim.Adam",
        "lr": 1e-3,
        "eps": 1e-5,
    },
}

### Launching tensorboard to visualize the results

In [8]:
setup_tensorboard("./outputs/tblogs")

Reusing TensorBoard on port 6006 (pid 752), started 22 days, 3:30:11 ago. (Use '!kill 752' to kill it.)

In [9]:
from torch.utils.tensorboard import SummaryWriter

2024-10-22 20:51:48.508065: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-22 20:51:48.559228: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-22 20:51:48.573950: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-22 20:51:48.682962: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-22 20:51:49.918838: W tensorflow/compiler/tf2

In [10]:
cfg = OmegaConf.create(params)
ppo_kl = PPOPenalty(cfg)
run_ppo_penalty(ppo_kl)

  0%|          | 0/5000 [00:00<?, ?it/s]

In [12]:
ppo_kl.visualize_best()

Video of best agent recorded in outputs/CartPole-v1/ppo-kl-S12-b5.0_20241022-205151/best_agent.mp4
Moviepy - Building video /home/chen_guanyu/M2A/M2A_RLD/outputs/CartPole-v1/ppo-kl-S12-b5.0_20241022-205151/best_agent.mp4.
Moviepy - Writing video /home/chen_guanyu/M2A/M2A_RLD/outputs/CartPole-v1/ppo-kl-S12-b5.0_20241022-205151/best_agent.mp4



Moviepy - Done !
Moviepy - video ready /home/chen_guanyu/M2A/M2A_RLD/outputs/CartPole-v1/ppo-kl-S12-b5.0_20241022-205151/best_agent.mp4
